In [1]:
import numpy as np
import pandas as pd
import math
import operator

from datetime import datetime

# Loading the data

For this problem '*Uber Pickups*' dataset was chosen. It represents the data for different New York Uber pickups.
Our goal is, by analysing this dataset, to predict the amount of pickups for each borough presented.

In [8]:
X_data = pd.read_csv('data/trainX.csv')
Y_data = pd.read_csv('data/trainY.csv')

In [9]:
X_data.head()

,id,pickup_dt,borough,spd,vsb,temp,dewp,slp,pcp01,pcp06,pcp24,sd,hday
0,26112,2015-06-12 16:00:00,NaN,6.0,9.1,82.0,60.0,1014.2,0.0,0.00,0.240000,0.000000,N
1,26896,2015-06-17 12:00:00,EWR,7.0,10.0,72.0,58.0,1018.9,0.0,0.00,0.270000,0.000000,N
2,2263,2015-01-15 08:00:00,Queens,3.0,10.0,27.0,17.0,1021.6,0.0,0.00,0.000000,0.000000,N
3,10453,2015-03-07 15:00:00,Brooklyn,7.0,9.1,24.0,7.0,1024.3,0.0,0.29,0.551667,18.583333,N
4,19043,2015-04-30 08:00:00,EWR,9.0,10.0,49.0,40.0,1006.7,0.0,0.00,0.050000,0.000000,N


For a solution using scikit-learn we considered 3 types of classifiers: *RandomForest*, *DecisionTree* and *LogisticRegression*.

(See solution using scikit-learn for details)

The following results (in points) were achieved by fitting (after applying Grid Search with different parameters on those classifiers (specific solvers were chosen)):

*   RandomForest (entropy) - **1246401.3858999142**
*   DecisionTree (gini) - **1245676.0787156357**
*   LogisticRegression (newton-cg) - **1193406.0331829898**

According to these results, the best algorithm to apply on our dataset (between those 3 been tried) is ***RandomForest***.
It will be implemented in this work to be then applied.

# Preparing the data

First, we don't need a pickup **id**.

In [10]:
X = pd.DataFrame(X_data, copy=True)
del X['id']

Second, we'll transform **pickup_dt** feature into *numeric* type (as done in a solution using scikit-learn).

In [11]:
X['pickup_dt'] = X['pickup_dt'].apply(
    lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").time()
).apply(
    lambda x: x.second+x.minute*60+x.hour*3600
)

Third, let's transform **holiday** char typed feature into *binary* typed

In [12]:
X['hday'] = X['hday'].apply(lambda x: 0 if x == 'N' else 1)

In [13]:
X.head()

,pickup_dt,borough,spd,vsb,temp,dewp,slp,pcp01,pcp06,pcp24,sd,hday
0,57600,NaN,6.0,9.1,82.0,60.0,1014.2,0.0,0.00,0.240000,0.000000,0
1,43200,EWR,7.0,10.0,72.0,58.0,1018.9,0.0,0.00,0.270000,0.000000,0
2,28800,Queens,3.0,10.0,27.0,17.0,1021.6,0.0,0.00,0.000000,0.000000,0
3,54000,Brooklyn,7.0,9.1,24.0,7.0,1024.3,0.0,0.29,0.551667,18.583333,0
4,28800,EWR,9.0,10.0,49.0,40.0,1006.7,0.0,0.00,0.050000,0.000000,0


Now let's normalize the data. We will use numpy's *linalg.norm()* function. It won't be implemented here as it's big enough, and you always can take a look at the source code via [this link](https://github.com/numpy/numpy/blob/v1.15.1/numpy/linalg/linalg.py#L2203-L2440).

In [14]:
def normalize(v):
  norm = np.linalg.norm(v)
  if norm == 0:
    return v
  return v / norm

In [15]:
for column_name in ['pickup_dt', 'spd', 'vsb', 'temp', 'dewp', 
                    'slp', 'pcp01', 'pcp06', 'pcp24', 'sd']:
  X[column_name] = normalize(X[column_name])

In [16]:
X.head()

,pickup_dt,borough,spd,vsb,temp,dewp,slp,pcp01,pcp06,pcp24,sd,hday
0,0.007780,NaN,0.005580,0.006513,0.010413,0.010504,0.006530,0.0,0.000000,0.006675,0.000000,0
1,0.005835,EWR,0.006510,0.007157,0.009143,0.010153,0.006561,0.0,0.000000,0.007509,0.000000,0
2,0.003890,Queens,0.002790,0.007157,0.003429,0.002976,0.006578,0.0,0.000000,0.000000,0.000000,0
3,0.007293,Brooklyn,0.006510,0.006513,0.003048,0.001225,0.006595,0.0,0.019599,0.015343,0.023457,0
4,0.003890,EWR,0.008371,0.007157,0.006222,0.007002,0.006482,0.0,0.000000,0.001391,0.000000,0


As it was done in a solution using scikit-learn, let's apply *one-hot encoding* for transforming **borough** feature data:

In [17]:
set(X['borough'].values)

{'Bronx', 'Brooklyn', 'EWR', 'Manhattan', 'Queens', 'Staten Island', nan}

In [18]:
for value in set(X['borough'].values):
    new_value = 'borough_{}'.format(value)
    X[new_value] = (X['borough'] == value)

In [19]:
del X['borough']
del X['borough_nan']
X.head()

,pickup_dt,spd,vsb,temp,dewp,slp,pcp01,pcp06,pcp24,sd,hday,borough_EWR,borough_Staten Island,borough_Bronx,borough_Queens,borough_Manhattan,borough_Brooklyn
0,0.007780,0.005580,0.006513,0.010413,0.010504,0.006530,0.0,0.000000,0.006675,0.000000,0,False,False,False,False,False,False
1,0.005835,0.006510,0.007157,0.009143,0.010153,0.006561,0.0,0.000000,0.007509,0.000000,0,True,False,False,False,False,False
2,0.003890,0.002790,0.007157,0.003429,0.002976,0.006578,0.0,0.000000,0.000000,0.000000,0,False,False,False,True,False,False
3,0.007293,0.006510,0.006513,0.003048,0.001225,0.006595,0.0,0.019599,0.015343,0.023457,0,False,False,False,False,False,True
4,0.003890,0.008371,0.007157,0.006222,0.007002,0.006482,0.0,0.000000,0.001391,0.000000,0,True,False,False,False,False,False


Let's prepare the vector of answers: 

In [20]:
Y_data.head()

,id,pickups
0,26112,2
1,26896,0
2,2263,210
3,10453,543
4,19043,0


In [21]:
merged = pd.merge(X_data, Y_data, on=['id'])
Y = merged['pickups'].values

Now let's split our training dataset into **N** pieces. We will then train **N** *decision trees* on each subset respectively. This will allow us to create an *ensemble* to improve results and minimize the possibility of overfitting.

We'll try to take **N** = 10.

In [22]:
X_copy = pd.DataFrame(X, copy=True)
subsets = []
amount = int(len(X_copy) / 10)

for x in range(0, 10):
    subsets.append(X_copy.sample(n=amount))
    X_copy.drop(subsets[x].index, inplace=True)

Now we have **10** random **subsets** formed from the original set. The next step is to implement the classifier. 

# Implementing Decision Tree (entropy)

*GridSearch* that was used in a solution using scikit-learn showed that *entropy-based* solver gives better results for our problem than *gini-based*. So ***entropy-based*** solver will be used here.

In [23]:
def entropy(data, attr):
    ser = data.groupby(attr).size().to_dict()
    entries = len(data)
    entropy = 0.0  # default value
    for key in ser.keys():
        probability = float(ser[key])/entries
        entropy -= probability * math.log(probability,2)
    return entropy

In [24]:
entropy(X, 'borough_Bronx')

0.6074703375428505

In [25]:
def split(data, colname, value):
  return data.loc[data[colname] == value, data.columns.drop(colname)]

In [1]:
def choose(data):
    minimum_entropy = entropy(data, data.columns[0])
    best_attr = -1
    
    values_map = {col: data.groupby(col).size().to_dict() for col in data.columns}
    
    entropies = []
    
    for attr in values_map:
        new_entropy = 0.0
        for value in values_map[attr]:
            new_data = split(X_copy, attr, value)
            probability = new_data.shape[0]/float(data.shape[0])
            new_entropy += probability * entropy(data, attr)
        entropies.append(dict(attr=attr, info_amount=new_entropy))
    
    print(entropies)
    best_attr = min(entropies, key=operator.itemgetter('info_amount'))
    print(f'best attribute is now {best_attr}')
    return best_attr['attr']

In [2]:
X_copy = pd.DataFrame(X, copy=True)

NameError: name 'pd' is not defined

In [3]:
choose(X_copy)

NameError: name 'X_copy' is not defined

In [29]:
def majority(classset):
    count = {}
    for attr in classset:
        if vote not in count.keys(): count[vote] = 0
        count[vote] += 1
    sorted_class_count = sorted(count.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sorted_class_count[0][0]

In [4]:
def tree(data, labels):
    if data.shape[0] == 1:
        return data.loc[:0]
    if data.columns.size == 1:
        return majority(data)
    best_feat = choose(data)
    print(labels)
    the_tree = {best_feat:{}}
    labels.remove(best_feat)
    print(best_feat)
    feat_values = data[best_feat]
    unique_vals = feat_values.unique()
    for value in unique_vals:
      print(f'\n {value} ({len(labels)} feats remaining)')
      sublabels = labels.copy()
      the_tree[best_feat][value] = tree(split(data, best_feat, value), sublabels)
    return the_tree

In [31]:
X_copy = pd.DataFrame(X, copy=True)

In [32]:
tree(X_copy, set(X_copy))

[{'attr': 'pickup_dt', 'info_amount': 4.5846038836929575}, {'attr': 'spd', 'info_amount': 4.169446478614493}, {'attr': 'vsb', 'info_amount': 2.194475411872334}, {'attr': 'temp', 'info_amount': 6.679872313241815}, {'attr': 'dewp', 'info_amount': 6.753885221566447}, {'attr': 'slp', 'info_amount': 8.196754732000358}, {'attr': 'pcp01', 'info_amount': 0.8983056198135245}, {'attr': 'pcp06', 'info_amount': 2.0615385457203335}, {'attr': 'pcp24', 'info_amount': 3.602350900945817}, {'attr': 'sd', 'info_amount': 3.0060684116839567}, {'attr': 'hday', 'info_amount': 0.2319152867402135}, {'attr': 'borough_EWR', 'info_amount': 0.6100552544042391}, {'attr': 'borough_Staten Island', 'info_amount': 0.6122001666595638}, {'attr': 'borough_Bronx', 'info_amount': 0.6074703375428505}, {'attr': 'borough_Queens', 'info_amount': 0.6096252715147293}, {'attr': 'borough_Manhattan', 'info_amount': 0.6068222221365697}, {'attr': 'borough_Brooklyn', 'info_amount': 0.6036789783978949}]
best attribute is now {'attr': 'h

KeyboardInterrupt: 

In [34]:
entropy_map = {}
for feat in set(X_copy):
  entropy_map[feat] = entropy(X_copy, feat)

In [35]:
entropy_map

{'hday': 0.2319152867402135,
 'temp': 6.679872313241805,
 'vsb': 2.1944754118723346,
 'borough_EWR': 0.6100552544042391,
 'borough_Staten Island': 0.6122001666595638,
 'sd': 3.006068411683965,
 'borough_Bronx': 0.6074703375428505,
 'pcp01': 0.8983056198135246,
 'pcp24': 3.602350900945828,
 'borough_Manhattan': 0.6068222221365697,
 'pickup_dt': 4.5846038836929575,
 'slp': 8.196754732000342,
 'borough_Queens': 0.6096252715147293,
 'dewp': 6.753885221566462,
 'borough_Brooklyn': 0.6036789783978949,
 'pcp06': 2.0615385457203352,
 'spd': 4.169446478614493}